## Deep Learning Basics in PyTorch
### Building a Basic Feedforward Neural Network
A feedforward neural network, also known as a multi-layer perceptron (MLP), is a basic type of neural network where information moves in only one direction—it is fed forward from the input layer, through any number of hidden layers, to the output layer. There are no cycles or loops in the network.

#### Building a Feedforward Neural Network with PyTorch
With PyTorch, we can easily build a feedforward neural network by defining a class that inherits from torch.nn.Module. Let's start with a simple network that has one input layer, one hidden layer, and one output layer.

First, we need to import the torch module and the torch.nn module, which contains the building blocks needed to build neural networks:

In [1]:
import torch
import torch.nn as nn

c:\Users\AB012DH\Anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\AB012DH\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\AB012DH\Anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


Next, we define our neural network:

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(10, 10)  # 10 input neurons, 10 output neurons
        self.fc2 = nn.Linear(10, 1)   # 1 output neuron

    def forward(self, x):
        x = torch.relu(self.fc1(x))  # ReLU activation function on first layer
        x = self.fc2(x)              # Output layer
        return x

Here's a breakdown of the code:

* We create a class Net that inherits from nn.Module.
* In the constructor (_ _ init _ _), we define two fully connected layers (fc1 and fc2). nn.Linear is used to apply a linear transformation to the incoming data: y = x*A^T + b.
* forward(x) is a method that implements the forward propagation of the input x. torch.relu is the Rectified Linear Unit (ReLU) activation function that is applied elementwise.

We can now create an instance of the neural network:

In [3]:
net = Net()
print(net)

Net(
  (fc1): Linear(in_features=10, out_features=10, bias=True)
  (fc2): Linear(in_features=10, out_features=1, bias=True)
)


Now that we have defined the network architecture, the next step would be to define a loss function and an optimizer, pass the input data through the network, calculate the loss, backpropagate the error, and update the weights. These steps are typically performed in a training loop, which we will discuss in a bit.

### Training a Feedforward Neural Network
Training a neural network involves iterating over a dataset of input samples and adjusting the weights of the network to minimize a loss function.

Let's continue with the feedforward neural network we built in the previous lesson. We'll use a simple regression problem where the task is to predict a single target value from 10 input features. We'll create an artificial dataset using torch.randn() for this purpose.

#### Preparing the Data
Let's generate a simple synthetic dataset:

In [8]:
input_features = torch.randn(100, 10)  # 100 samples, 10 features each
target_values = torch.randn(100, 1)    # 100 target values

This creates two tensors: input_features containing 100 samples each with 10 features, and target_values containing 100 corresponding target values. The values are randomly generated.

#### Defining a Loss Function and an Optimizer
We'll use Mean Squared Error (MSE) as our loss function—it's a common choice for regression problems. For optimization, we'll use Stochastic Gradient Descent (SGD). PyTorch provides these as torch.nn.MSELoss and torch.optim.SGD respectively:

In [9]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)  # net is our network

The optimizer adjusts the weights of the network based on the gradients computed during backpropagation.
#### Training the Network
Training involves a loop over epochs (iterations over the whole dataset). In each epoch, we:
1. Run the forward pass to get the outputs from the input features,
2. Calculate the loss between the outputs and the target values,
3. Backpropagate the error through the network,
4. Use the optimizer to adjust the weights.

In [10]:
for epoch in range(100):  # Number of times to loop over the dataset
    # Forward pass
    outputs = net(input_features)
    
    # Compute loss
    loss = loss_fn(outputs, target_values)
    
    # Zero the gradients
    optimizer.zero_grad()
    
    # Perform backward pass (backpropagation)
    loss.backward()
    
    # Update the weights
    optimizer.step()
    
    # Print loss every 10 epochs
    if epoch % 10 == 0:
        print(f'Epoch: {epoch}, Loss: {loss.item()}')

Epoch: 0, Loss: 1.185062289237976
Epoch: 10, Loss: 1.1039198637008667
Epoch: 20, Loss: 1.0623257160186768
Epoch: 30, Loss: 1.0384596586227417
Epoch: 40, Loss: 1.0231518745422363
Epoch: 50, Loss: 1.0120700597763062
Epoch: 60, Loss: 1.0031964778900146
Epoch: 70, Loss: 0.9956872463226318
Epoch: 80, Loss: 0.9888506531715393
Epoch: 90, Loss: 0.9825147986412048


After running this code, you'll see the loss decreasing over epochs, which means our model is learning to fit the input features to the target values.

### Evaluating a Feedforward Neural Network
After training a neural network, it's crucial to evaluate its performance on a separate test dataset that the model has never seen during training. This helps to determine if the model is generalizing well or just memorizing the training data (overfitting).
We will continue from the feedforward neural network model we trained in the previous lesson.

#### Preparing the Test Data
Similar to the training data, we'll create synthetic test data:

In [11]:
test_features = torch.randn(50, 10)  # 50 samples, 10 features each
test_targets = torch.randn(50, 1)    # 50 target values

#### Evaluating the Model
Now, let's evaluate our model on this test data. While doing so, we'll switch off PyTorch's autograd engine using torch.no_grad(). This reduces memory usage and speeds up computations but is unable to perform backpropagation.

In [12]:
with torch.no_grad():  # Switch off gradients
    test_outputs = net(test_features)

#### Calculating the Loss
To calculate the loss between our model's predictions and the true values:

In [13]:
test_loss = loss_fn(test_outputs, test_targets)
print(f'Test Loss: {test_loss.item()}')

Test Loss: 1.1997207403182983


This will print the test loss value. In a perfect model, this would be 0, but in practice it's always higher. Ideally, this test loss should be close to the training loss. If the test loss is much higher, it might be a sign that the model is overfitting.

### Backpropagation and Computational Graphs
Backpropagation is a critical process in training deep learning models. It is a method used to calculate the gradient of the loss function with respect to each weight in the model, which is then used to update the weights. Backpropagation is made possible in PyTorch through the construction of computational graphs during the forward pass. In this lesson, we will discuss computational graphs and backpropagation in more detail.

#### What is a Computational Graph?
A computational graph is a directed graph where the nodes correspond to operations or variables. Variables can be input tensors, parameters (such as weights and biases), or intermediate values. Operations can be any mathematical operations, such as addition and multiplication, or more complex ones like activation functions.

When a forward pass is made in a neural network, a computational graph is dynamically created in PyTorch. This graph is used to compute the gradients during the backward pass, which is a fundamental step in optimizing the network.

#### Backpropagation and Computational Graphs
Let's consider a simple example to understand how PyTorch constructs a computational graph and how backpropagation is performed. Let's take two tensors a and b, which are parameters of the model and require gradient computation, and a simple equation f = (a * b) + b.

In [14]:
import torch

# Initialize tensors
a = torch.tensor([2.], requires_grad=True)
b = torch.tensor([3.], requires_grad=True)

# Compute outputs
f = (a * b) + b

# Compute gradients
f.backward()

# Print gradients
print(a.grad)  # df/da
print(b.grad)  # df/db

tensor([3.])
tensor([3.])


In this example, a computational graph is created in the following way:

Nodes are created for the tensors a and b.
When f = (a * b) + b is computed, nodes are created for the operations * and +, and for the output f. Edges are created to connect these operations.
During the backward pass (f.backward()), the gradients of the output with respect to each tensor that has requires_grad=True are computed using this computational graph.
The returned gradients are the derivatives of f with respect to a and b and are stored in the .grad property of the respective tensors.

In more complex models like deep neural networks, the computational graph can be far more intricate, but the idea remains the same. During backpropagation, the gradients of the loss with respect to the weights are computed and used to update the weights.

### Building Neural Networks using nn.Module
We've already touched on the usage of nn.Module in PyTorch when building our simple feedforward network, but it's worth diving deeper into how it works and why it's such a fundamental part of structuring models in PyTorch.

nn.Module is a base class for all neural network modules in PyTorch. Your models should also subclass this class, which allows you to utilize powerful PyTorch functionalities. Modules can contain other Modules, allowing to nest them in a tree structure.

#### Building a Network with nn.Module
Let's expand our basic feedforward network into a multi-layer one using nn.Module:

In [16]:
import torch.nn as nn

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        # Input layer to hidden layer
        self.hidden = nn.Linear(10, 20) 

        # Hidden layer to output layer
        self.output = nn.Linear(20, 1)  

        # Activation function
        self.relu = nn.ReLU()             

    def forward(self, x):
        x = self.hidden(x)  # Pass data through hidden layer
        x = self.relu(x)    # Apply activation function
        x = self.output(x)  # Pass data through output layer
        return x

In the Net class defined above, we first define the layers of our network in the constructor __init__(). We have an input to hidden layer connection (self.hidden), an activation function (self.relu), and a hidden to output layer connection (self.output).

Then, in the forward function, we define the path that the input tensor will take through the network.

#### Why nn.Module?
* Parameter management: Once layers are defined as members of nn.Module (in our example, self.hidden and self.output), all parameters of these layers are registered in the module. You can access all parameters of your network using the parameters() or named_parameters() methods.
* Device management: You can recursively move all parameters of the network to a device (CPU or GPU) by calling .to(device) on the network. This saves you the trouble of moving each tensor individually.
* Easy to use with optimizers: You can pass the parameters of a network (retrieved through net.parameters()) to an optimizer, and it will take care of the parameter updates for you.
* Helper methods and functionalities: There are many other functionalities provided by nn.Module such as the ability to save and load the model, ability to set the network in training or evaluation mode (net.train(), net.eval()).

### Debugging PyTorch models, Using torchviz to visualize networks
Debugging PyTorch models and visualizing the computational graph of your model is an essential part of understanding and developing deep learning models. PyTorch does not come with a built-in tool for this, but a popular third-party library, torchviz, can be used to create a visualization of your model.

#### Debugging PyTorch Models
Debugging PyTorch models can be done in the same way as debugging any Python program, using Python's built-in debugging tools. The most popular is pdb, the Python debugger. You can insert breakpoints into your code using pdb.set_trace(), which allows you to step through your code and inspect variables.

For example, during the forward pass of your model:

In [17]:
import pdb

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc = nn.Linear(10, 1)

    def forward(self, x):
        pdb.set_trace()  # Set breakpoint here
        x = self.fc(x)
        return x

When the forward method is called, the program will pause at pdb.set_trace(). You can then print the values of variables, step through the code, and more. To continue execution, type c and hit enter.

#### Visualizing Networks with torchviz
torchviz is a PyTorch specific package that creates directed graphs of PyTorch execution traces and computational graphs. It's handy to understand the operations and tensor transformations that take place during the forward and backward pass.

To use torchviz, you first need to install it, and also be sure to download the installer from https://graphviz.org/download/:

In [19]:
!pip install torchviz

^C


  Created wheel for torchviz: filename=torchviz-0.0.2-py3-none-any.whl size=4150 sha256=bd5ffca33742cc5299a3b7717641b4d91e1502d011039b51aec1b9eb8aa6e663
  Stored in directory: c:\users\ab012dh\appdata\local\pip\cache\wheels\29\65\6e\db2515eb1dc760fecd36b40d54df65c1e18534013f1c037e2e
Successfully built torchviz


Here is how to use it to visualize the computation graph of a simple model:

In [1]:
import torch
from torch import nn
from torchviz import make_dot

model = nn.Sequential()
model.add_module('W0', nn.Linear(8, 16))
model.add_module('tanh', nn.Tanh())
model.add_module('W1', nn.Linear(16, 1))

x = torch.randn(1,8)

# Forward pass
y = model(x)

# Create a visualization of the forward computation graph
make_dot(y, params=dict(model.named_parameters())).render("rnn_torchviz", format="png")

c:\Users\AB012DH\Anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\AB012DH\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\AB012DH\Anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


'rnn_torchviz.png'